# Prediction of drug target interaction for drug discovery

### Introduction


<div style="text-align: justify"> 

La prédiction de l'intéraction entre une molécule et une cible thérapeutique est un élément essentiel dans le processus de découverte de médicaments, cette étape est cruciale pour développer de nouveaux médicaments ou améliorer ceux déjà existants. Le machine learning est devenu un outil puissant dans la


IC50 and pIC50 are related metrics that measure the potency of a compound's interaction with a target protein. However, they are not the same thing.

- IC50 represents the concentration of a compound that is required to inhibit the activity of the target protein by 50%. It is typically measured in units of molarity, such as nanomolar (nM) or micromolar (μM). A lower IC50 value indicates that a compound is more potent, as it can inhibit the target protein at a lower concentration.

- On the other hand, pIC50 is a logarithmic transformation of IC50, and is defined as -log(IC50). pIC50 values are typically reported in units of -log(molarity), such as -log(nM) or -log(μM). A higher pIC50 value indicates that a compound is more potent, as it can inhibit the target protein at a lower concentration.

The relationship between IC50 and pIC50 can be expressed mathematically as follows: 

$$pIC50 = -log(IC50)$$

</div>


This notebook aims to provide a baseline for the challenge, by proposing a simple pipeline based on ECFP fingerprints combined with a Random Forest algorithm. The objective is to predict the pIC50 of various drugs on the VEGFR2 receptors. You have at your disposal a training dataset and a test dataset on which to make your predictions.

### Imports

In [19]:
# Linear algebra and data handling
import numpy as np
import pandas as pd

# RDKit
import rdkit 
from rdkit import Chem
from rdkit.Chem import AllChem

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

# Viz
import matplotlib.pyplot as plt

### VEGFR2

First, we need to load the dataset. Do not hesitate to do more exploratory data analysis on your own !

In [20]:
# Loading the train and test csv files
train = pd.read_csv('../data/X_train.csv')
test = pd.read_csv('../data/X_test.csv')

y_train = pd.read_csv('../data/y_train.csv')['y']

# You can check the shapes of the train and test sets
print(train.shape, test.shape)

# You can check what it looks like
train.head()

(4400, 2) (2934, 2)


,id,smiles
0,0,CNC(=O)c1ccccc1Sc1ccc2c(C#Cc3cccc(NCCOC)c3)n[n...
1,1,CC(C)(C)c1ccc(Nc2nnc(-c3cnccc3CCc3ccncc3)o2)cc1
2,2,CN1CCN(CCCn2ccc(-c3cnc4c(-c5ccsc5)cnn4c3)cc2=O...
3,3,CCn1c2ccc(NC(=O)Nc3ccc(OC)cc3)cc2c2c3c(c4c(c21...
4,4,Cc1ccc(C(=O)Nc2cccc(C(C)C)c2)cc1N1CCc2ncncc2C1


As you can see, molecules are encoded using the SMILES notation. To be understandable by the rdkit package, we need to convert them to "Mol" type.

In [21]:
# Convert SMILES to mol files
train_mols = [AllChem.MolFromSmiles(smile) for smile in train['smiles']]
test_mols = [AllChem.MolFromSmiles(smile) for smile in test['smiles']]

type(train_mols[0])

rdkit.Chem.rdchem.Mol

After converting SMILES to Mol, we can compute fingerprints. For this example, we use the Morgan fingerprints, a popular type of circular fingerprints, but many others exist. 

In [22]:
# Convert Mol to fingerprints
train_fps = np.array([AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in train_mols])
test_fps = np.array([AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in test_mols])

print(train_fps.shape, test_fps.shape)

(4400, 2048) (2934, 2048)


We will be using scikit-learn implementation of Random Forest as predictive model. We can now define our new features.

In [23]:
X_train = train_fps
X_test = test_fps

Cross-validation is a good practice to estimate the "true" performance of a model on unseen data. If the cross-validation scores are satisfying, we can then train the model on the entire dataset and predict the test set. For this introduction we use the default parameters of the Random Forest, but keep in mind they might need to be optimized for the challenge. You are encouraged to try other models.

This run can take a few minutes depending on your computer. 

In [24]:
model = RandomForestRegressor(n_jobs=-1)
cv = cross_validate(estimator=model, 
                    X=X_train, 
                    y=y_train, 
                    cv=5, 
                    scoring=["neg_median_absolute_error", "neg_root_mean_squared_error"],
                    n_jobs=-1,
                    verbose=0)

print(f"Mean MAE over 10 folds = {-cv['test_neg_median_absolute_error'].mean():.4f}")
print(f"Mean RMSE over 10 folds = {-cv['test_neg_root_mean_squared_error'].mean():.4f}")

Mean MAE over 10 folds = 0.4269
Mean RMSE over 10 folds = 0.7898


We can now predict X_test and make a submission file (csv) that will later be uploaded on the challenge's website. 

In [25]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [26]:
submission_df = pd.DataFrame()
submission_df['id'] = test['id']
submission_df['y'] = y_pred

submission_df.to_csv('../data/y_benchmark.csv', index=False)